In [1]:
!mkdir build
%cd build
!cmake ../
!make

mkdir: cannot create directory ‘build’: File exists
/home/aseal/Documents/invint/build
-- Configuring done
-- Generating done
-- Build files have been written to: /home/aseal/Documents/invint/build
[  3%] Built target dtnode
[  5%] Performing update step for 'antlr4_runtime'
Ignoring path runtime/Cpp/runtime/thirdparty/utfcpp/
Current branch master is up to date.
Already up to date!
[  6%] Performing configure step for 'antlr4_runtime'
loading initial cache file /home/aseal/Documents/invint/build/antlr4_runtime/tmp/antlr4_runtime-cache-.cmake
CMake Warning at CMakeLists.txt:7 (message):
  Build type not set, falling back to Release mode.

   To specify build type use:
   -DCMAKE_BUILD_TYPE=<mode> where <mode> is Debug or Release.


-- Building without demo. To enable demo build use: -DWITH_DEMO=True
CMake Deprecation Warning at CMakeLists.txt:31 (CMAKE_POLICY):
  The OLD behavior for policy CMP0054 will be removed from a future version
  of CMake.

  The cmake-policies(7) manual explai

In [2]:
import time
import subprocess
import numpy as np
import json
import pandas as pd

In [3]:
def time1(query):
    q = open("query.txt","w")
    q.write(query + "\n")
    q.close()
    times = []
    out = ""
    for i in range(5):
        d1 = time.process_time()
        try:
            sp = subprocess.run("./main2 --model=dt --file=dt.txt < query.txt", shell = True, capture_output = True, timeout = 60)
            out = sp.stdout.splitlines()[2]
        except:
            print("timeout with query " + query)
            times = [10,10,10,10,10]
            out = "timeout"
            break
        #timeout!!!
        d2 = time.process_time()
        times.append(d2-d1)
    return (times , out)

def random_constant(query , dt_size):
    ans = ""
    idx = 1
    for i in range(len(query[0])):
        if (query[0][i] == 'C'):
            ans+="[ "
            for j in range(dt_size):
                if (j):
                    ans += " , "
                val = query[idx][j]
                if (val == 2): val = '?'
                else: val = str(val)
                ans += val
            ans += " ] "
            idx+=1
        else:
            ans+=query[0][i]
    return ans

def generate_query(squeries , nvars , rng):
    query = ""
    isnot = rng.integers(2)
    if (isnot):
        query += " ~ ( "
    if (squeries == 1):
        
        if (nvars == 1):
            opt = rng.integers(3)
        else:
            opt = rng.integers(4)
        
        if (opt == 0):
            
            query += " P ( x" + str( rng.integers(nvars) + 1 ) + " ) "
        
        if (opt == 1):
            
            query += " C <= x" + str( rng.integers(nvars) + 1 ) + " "
        
        if (opt == 2):
            
            query += " x" + str( rng.integers(nvars) + 1 ) + " <= C "
            
        if (opt == 3):
            
            x1 = rng.integers(nvars) + 1
            x2 = rng.integers(nvars) + 1
            while (x2 == x1):
                x2 = rng.integers(nvars) + 1
            query += " x" + str(x1) + " <= x" + str(x2) + " "
        
    else:
        
        query += " ( "
        fs = rng.integers(squeries-1) + 1
        query += generate_query(fs , nvars , rng)
        
        if ( rng.integers(2) ):
            query += " ) V ( "
        else:
            query += " ) ^ ( "
            
        query += generate_query(squeries - fs , nvars , rng) + " ) "
    
    if (isnot):
        query += " ) "
    
    return query

def time_queries(dt_name , queries, rng):
    
    data = {}
    
    with open("../dtrees/" + dt_name) as json_file:
        data = json.load(json_file)
        
    with open('dt.txt', 'w') as outfile:
        json.dump(data, outfile)
    
    times = []
    timesr = []
    
    for q in queries:
        queryt = random_constant(q , len( data["feature_names"] ) )
        ctimes = time1( queryt )
        timep = ( sum(ctimes[0]) - max(ctimes[0]) - min(ctimes[0]))/3
        timesr.append( timep )
        finalarr = [queryt] + ctimes[0] + [ctimes[1]] + [timep]
        times.append(finalarr)
        
    df = pd.DataFrame( times , columns = ["query" , "t1" , "t2" , "t3" , "t4" , "t5" , "ans" , "tp"] )
    
    df.to_csv(dt_name + ".csv")
    
    return timesr

def test_trees(qsize, nvars , nqueries, seed):
    data = []
    rng = np.random.default_rng(seed)
    queries = []
    dims = [3, 5, 10, 15, 20, 25, 30]
    leaves =  [5, 10, 15, 20, 50, 100, 150, 200, 500, 1000, 2000]
    for i in range(nqueries):
        q = ""
        ex = rng.integers(2)
        for j in range(nvars):
            if (ex):
                q += "Exists x" + str(j + 1) + ", "
            else:
                q += "Forall x" + str(j + 1) + ", "
        q += generate_query(qsize , nvars , rng)
        nconstants = q.count('C')
        fquery = [q]
        for i in range(nconstants):
            c = []
            for j in range( max(dims) ):
                c.append( rng.integers(3) )
            fquery.append(c)
        queries.append(fquery)
    infop = [ ]
    for l in leaves:
        for d in dims:
            infop.append( [d , l ] +  time_queries( "dim_" + str(d) + "_nleaves_" + str(l) + ".json" , queries , rng) )
            print( str(l) + " " + str(d) + " " + str(sum(infop[len(infop)-1][2:])/nqueries) )
    
    qs = []
    for i in range(nqueries):
        qs.append( queries[i][0] )
    df = pd.DataFrame( infop , columns = ["dimension" , "leaves" ] + qs )
    df.to_csv("final_data.csv")
    

In [4]:
#test_trees(8,3,10,5)

In [15]:
def create_constant(max_s,rng):
    return rng.integers(3,size=max_s)

def replace_constants(q , cs, dim):
    nq = ""
    idx = 0
    for l in q:
        if l == 'C':
            nq += " [ "
            for j in range(dim):
                if (j): nq += " , "
                if (cs[idx][j] == 2): nq += " ? "
                else: nq += str(cs[idx][j])
            nq += " ] "
            idx+=1
        else: nq += l
    return nq

def load_tree(dt_name):
    with open("../dtrees/" + dt_name) as json_file:
        data = json.load(json_file)
        
    with open('dt.txt', 'w') as outfile:
        json.dump(data, outfile)
        
def queries_with_tree( dim, leaves, queries, constants ):
    load_tree("dim_" + str(dim) + "_nleaves_" + str(leaves) + ".json")
    n = len(queries)
    info = [ dim , leaves ]
    table = []
    
    for i in range(n):
        q = replace_constants(queries[i] , constants[i] , dim)
        tims =  time1( q )
        tp = (sum( tims[0] ) - max(tims[0]) - min(tims[0]) )/3
        info.append(tp)
        table.append( [ q ] + tims[0] + [ tims[1] , tp ]  )
    
    df = pd.DataFrame( table , columns = ["query" , "t1" , "t2" , "t3" , "t4" , "t5" , "ans" , "tp"] )
    
    df.to_csv( "dim_" + str(dim) + "_nleaves_" + str(leaves) + ".csv")
    
    return info
    

def run_queries(queries , seed):
    
    dims = [3, 5, 10, 15, 20, 25, 30]
    leaves =  [5, 10, 15, 20, 50, 100, 150, 200, 500, 1000, 2000]
    
    cs = []
    rng = np.random.default_rng()
    for query in queries:
        cc = []
        for j in range( query.count('C') ):
            cc.append( create_constant( max(dims) , rng ) )
        cs.append(cc)
    
    table = []
    
    
    for dim in dims:
        for leaf in leaves:
            table.append( queries_with_tree(dim , leaf , queries , cs) )
            
    df = pd.DataFrame( table , columns = ["dimension" , "leaves" ] + queries )
    df.to_csv("queries.csv")

In [16]:
run_queries(["Exists x, P(x)" , "Exists x, ~ ( P(x) )" , "Exists x, Exists y, ( P(x) ) ^ ( ~ ( P ( y ) ) )" , "ForAll x, ForAll y, (( P(x) ) V ( P(y) )) V  ( (x <= C) ^ (y <= C))"] , 10)